
# 데이터 통합 프로젝트

이 노트북은 Welcome과 Quickpoll 데이터를 통합하는 프로세스를 담고 있습니다.

## 프로젝트 개요
- **목적**: Welcome 파일 2개와 Quickpoll 파일 35개를 mb_sn을 기준으로 통합
- **데이터 소스**: `../data/raw/` 폴더의 Excel 파일들
- **통합 기준**: `mb_sn` 컬럼을 사용한 Outer Join
- **특징**: 
  - Welcome 파일: 첫 번째 행을 헤더로 사용 (header=0)
  - Quickpoll 파일: 두 번째 행을 헤더로 사용 (header=1)
  - '고유번호' 컬럼을 'mb_sn'으로 변경
  - Outer Join으로 모든 데이터 보존
  - 중복 컬럼 자동 정리

## 1. 필요한 라이브러리 불러오기

데이터 처리에 필요한 라이브러리들을 import합니다.

In [28]:
import pandas as pd
import glob  # 파일 경로를 쉽게 다룰 수 있게 해주는 라이브러리
import os    # 운영체제와 상호작용하는 라이브러리


## 2. 파일 경로 설정

데이터 파일들이 위치한 경로를 설정합니다.


In [29]:
# 경로 설정 및 파일 목록 불러오기
base_path = '../data/raw/'
welcome_path = os.path.join(base_path, 'Welcome/*.xlsx')
quickpoll_path = os.path.join(base_path, 'Quickpoll/*.xlsx')

welcome_files = glob.glob(welcome_path)
quickpoll_files = glob.glob(quickpoll_path)

print(f"Welcome 파일 경로: {welcome_path}")
print(f"Quickpoll 파일 경로: {quickpoll_path}")
print(f"Welcome 파일 개수: {len(welcome_files)}개")
print(f"Quickpoll 파일 개수: {len(quickpoll_files)}개")


Welcome 파일 경로: ../data/raw/Welcome/*.xlsx
Quickpoll 파일 경로: ../data/raw/Quickpoll/*.xlsx
Welcome 파일 개수: 2개
Quickpoll 파일 개수: 35개


## 3. 파일 목록 불러오기

glob을 사용하여 각 폴더의 Excel 파일들을 찾습니다.


In [30]:
# 파일 목록 확인
print("\n=== Welcome 파일 목록 ===")
for i, file in enumerate(welcome_files, 1):
    print(f"{i}. {os.path.basename(file)}")

print("\n=== Quickpoll 파일 목록 ===")
for i, file in enumerate(quickpoll_files, 1):
    print(f"{i}. {os.path.basename(file)}")



=== Welcome 파일 목록 ===
1. Welcome_2nd.xlsx
2. Welcome_1st.xlsx

=== Quickpoll 파일 목록 ===
1. qpoll_join_250106.xlsx
2. qpoll_join_250604.xlsx
3. qpoll_join_250716.xlsx
4. qpoll_join_250624.xlsx
5. qpoll_join_250326.xlsx
6. qpoll_join_250310.xlsx
7. qpoll_join_250605.xlsx
8. qpoll_join_250107.xlsx
9. qpoll_join_250116.xlsx
10. qpoll_join_250317.xlsx
11. qpoll_join_250710.xlsx
12. qpoll_join_250707.xlsx
13. qpoll_join_250204.xlsx
14. qpoll_join_250619.xlsx
15. qpoll_join_250224.xlsx
16. qpoll_join_250623.xlsx
17. qpoll_join_250620.xlsx
18. qpoll_join_250616.xlsx
19. qpoll_join_250704.xlsx
20. qpoll_join_250206.xlsx
21. qpoll_join_250617.xlsx
22. qpoll_join_250226.xlsx
23. qpoll_join_250123.xlsx
24. qpoll_join_250709.xlsx
25. qpoll_join_250610.xlsx
26. qpoll_join_250702.xlsx
27. qpoll_join_250714.xlsx
28. qpoll_join_250221.xlsx
29. qpoll_join_250626.xlsx
30. qpoll_join_250723.xlsx
31. qpoll_join_250627.xlsx
32. qpoll_join_250328.xlsx
33. qpoll_join_250703.xlsx
34. qpoll_join_250611.xlsx
35.

## 4. Welcome 파일 통합

Welcome 파일들을 먼저 통합하여 기본 DataFrame을 생성합니다.


In [31]:
# Welcome 파일 통합 (header=1 없이)
# Welcome 파일은 깨끗하다고 가정하고 그대로 읽습니다.
print("=== Welcome 파일 통합 시작 ===")
base_df = pd.read_excel(welcome_files[0])
print(f"기본 파일 로드: {os.path.basename(welcome_files[0])}")
print(f"기본 데이터 크기: {base_df.shape}")

# '고유번호' 컬럼을 'mb_sn'으로 변경
if '고유번호' in base_df.columns:
    base_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
    print("'고유번호' 컬럼을 'mb_sn'으로 변경했습니다.")

# 나머지 Welcome 파일들 통합
for file in welcome_files[1:]:
    print(f"병합 중: {os.path.basename(file)}")
    temp_df = pd.read_excel(file)
    print(f"  - 임시 데이터 크기: {temp_df.shape}")
    
    # '고유번호' 컬럼을 'mb_sn'으로 변경
    if '고유번호' in temp_df.columns:
        temp_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
    
    # Welcome 파일끼리는 outer merge로 모든 패널을 포함시킵니다.
    base_df = pd.merge(base_df, temp_df, on='mb_sn', how='outer')
    print(f"  - 통합 후 크기: {base_df.shape}")

print(f"--- Welcome 파일 통합 완료 (기준 데이터 크기: {base_df.shape}) ---")


=== Welcome 파일 통합 시작 ===
기본 파일 로드: Welcome_2nd.xlsx
기본 데이터 크기: (28744, 22)
병합 중: Welcome_1st.xlsx
  - 임시 데이터 크기: (27221, 5)
  - 통합 후 크기: (35240, 26)
--- Welcome 파일 통합 완료 (기준 데이터 크기: (35240, 26)) ---


## 5. Quickpoll 파일 통합

Quickpoll 파일들을 기본 DataFrame에 통합합니다.


In [35]:
# Quickpoll 파일 통합 (how='outer' 사용)
print("=== Quickpoll 파일 통합 시작 ===")
successful_merges = 0
skipped_files = 0

common_info_cols = ['구분', 'mb_sn', '성별', '나이', '지역', '설문일시']

for file in quickpoll_files:
    filename = os.path.basename(file).split('.')[0] # 파일 이름 (확장자 제외)
    print(f"병합 중: {filename}.xlsx")
    temp_df = pd.read_excel(file, header=1)
    
    if '고유번호' in temp_df.columns:
        temp_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
    
    if 'mb_sn' in temp_df.columns:
        # (1) '설문 답변' 컬럼만 골라냅니다.
        answer_cols = [col for col in temp_df.columns if col not in common_info_cols]
        
        # 💡 --- 핵심 수정 부분 ---
        # (2) 각 설문 답변 컬럼 이름 뒤에 파일 이름을 붙여 고유하게 만듭니다.
        rename_dict = {col: f"{col}_{filename}" for col in answer_cols}
        temp_df.rename(columns=rename_dict, inplace=True)
        
        # (3) 고유해진 설문 답변 컬럼 목록을 다시 가져옵니다.
        unique_answer_cols = list(rename_dict.values())
        
        # (4) 병합할 데이터는 'mb_sn'과 '고유해진 설문 답변'만 포함시킵니다.
        merge_target_df = temp_df[['mb_sn'] + unique_answer_cols]
        
        # (5) 최종 병합을 수행합니다.
        base_df = pd.merge(base_df, merge_target_df, on='mb_sn', how='outer')
        # --- 수정 완료 ---
    else:
        print(f"  [경고] 병합 키가 없어 건너뜁니다.")

print(f"\n--- Outer 병합 완료 (중복 컬럼 존재 상태) ---")
print(f"성공적으로 통합된 파일: {successful_merges}개")
print(f"건너뛴 파일: {skipped_files}개")
print(f"데이터 크기: {base_df.shape}")


=== Quickpoll 파일 통합 시작 ===
병합 중: qpoll_join_250106.xlsx
병합 중: qpoll_join_250604.xlsx
병합 중: qpoll_join_250716.xlsx


: 

## 6. 중복 컬럼 정리

병합 과정에서 생긴 _x, _y 컬럼들을 하나로 합칩니다.


In [ ]:

# 중복 컬럼 정리 (정보 보완)
# 병합 과정에서 생긴 _x, _y 컬럼들을 하나로 합칩니다.
# 예: '성별_x', '성별_y' -> '성별'
common_cols_to_clean = ['성별', '나이', '지역', '구분', '설문일시']

print("=== 중복 컬럼 정리 시작 ===")
for col in common_cols_to_clean:
    col_x, col_y = f'{col}_x', f'{col}_y'
    if col_x in base_df.columns and col_y in base_df.columns:
        # _y 컬럼의 값으로 _x 컬럼의 빈칸(NaN)을 채운 뒤,
        # 최종적으로 정리된 컬럼 하나만 남깁니다.
        base_df[col] = base_df[col_y].fillna(base_df[col_x])
        base_df.drop(columns=[col_x, col_y], inplace=True)
        print(f"  - {col} 컬럼 정리 완료")

print("--- 중복 컬럼 정리 완료 ---")
print(f"정리 후 데이터 크기: {base_df.shape}")


=== 중복 컬럼 정리 시작 ===
--- 중복 컬럼 정리 완료 ---
정리 후 데이터 크기: (35240, 26)


## 7. 최종 결과 확인

통합 및 정리된 데이터의 정보를 확인합니다.


In [ ]:
print("\n🎉 최종 데이터 통합 및 정리 완료!")
print("=" * 50)
print("--- 통합 데이터 정보 ---")
base_df.info()

print("\n--- 통합 데이터 크기 ---")
print(f"최종 데이터 크기: {base_df.shape[0]}개의 행, {base_df.shape[1]}개의 열")

print("\n--- 통합 데이터 미리보기 ---")
display(base_df.head())


## 8. 데이터 저장 (선택사항)

통합된 데이터를 CSV 파일로 저장할 수 있습니다.


In [ ]:
# # 최종 결과물 저장
# output_path = '../data/processed/01_consolidated_data.csv'
# base_df.to_csv(output_path, index=False, encoding='utf-8-sig')
# print(f"통합된 데이터가 '{output_path}' 경로에 저장되었습니다.")
# print(f"저장된 파일 크기: {base_df.shape}")
